In [1]:
# Check if notebook is running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

In [ ]:
if IN_COLAB:
    # Use the token from Github to clone the PRECISE repository with read/write access
    from IPython.display import clear_output; user="kudzaishezororo"; token=input();
    !git clone https://github.com/zororo2/pals.git

    clear_output()
    # Install relevant packages
    !pip install geehydro cartopy
    !pip install rioxarray
    !pip install mapclassify
    clear_output()

In [ ]:
import os
import folium, cartopy, mapclassify
import geehydro
import geopandas as gpd
import numpy as np, pandas as pd
import cartopy.crs as ccrs
import ee
import rioxarray as rio, xarray as xr
import geemap
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

In [ ]:
# Authenticate and initialise Earth Engine API
try:
    ee.Initialize(project="ee-monthlyexposures")
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project="ee-monthlyexposures")

In [ ]:
#Check if the repo is cloned successfully
!ls /content/pals/shapefiles1 # Assuming the repository is cloned into the root of your Colab workspace

precise_villages1.gpkg	precise_villages.gpkg  shape.gitkeep


In [ ]:
# Replace 'path/to/your/file.shp' with the actual path
shapefile_path = 'path/to/your/file.shp'
#gdf1 = gpd.read_file('/content/pals/shapefiles1/precise_villages.gpkg', layer='kenya')
#gdf2 = gpd.read_file('/content/pals/shapefiles1/kenya_villages.gpkg')



# Load the GeoPackage layer
file_path = '/content/pals/shapefiles1/precise_villages1.gpkg'
layer_name = 'kenya'
gdf = gpd.read_file(file_path, layer=layer_name)

# Print the attribute table
print(gdf)


#gdf1.plot()



     OBJECTID  village_co          name    Field5        chu  latitude  \
0           1           0        Bambam      None      mwele -3.941202   
1           2           0     Bara mare  Baremare       buni -3.949008   
2           3      254001        BATANI      None    KOMBENI -3.889574   
3           4      254002  BATANI ASILI      None    KOMBENI -3.896055   
4           5           0        Bejana      None       Buni -3.948978   
..        ...         ...           ...       ...        ...       ...   
121       122      254073        TUNOLO      None  KAMKOMANI -3.871829   
122       123      254313     VISHAKANI      None  VISHAKANI -3.837140   
123       124      254074       VIZIANI      None  KAMKOMANI -3.891758   
124       125      254075          VUMA      None    SHANGIA -3.820871   
125       126      254316   WEIGHBRIDGE      None    MWARENI -3.834980   

     longitude  accuracy  BUFF_DIST  ORIG_FID  Shape_Leng  Shape_Area  \
0    39.571624    12.000       75.0   

In [ ]:
def generateImageCollection(exposure, country, dataset, facilities=False):

    path='shapefiles/precise_villages.gpkg'
    path=os.path.join(r'/content/precisehealthgeo', path) if IN_COLAB else path
    start='2018-11-01'
    end='2023-12-31'
    ### image processing functions ###
    ##################################

    # load the shapefile to geodataframe
    if facilities:
        gdf=gpd.read_file('/content/pals/shapefiles1/precise_villages.gpkg', layer='{}_health_facilities'.format(country))
        #gdf=gpd.read_file(path, layer='{}_health_facilities'.format(country))

        # gdf=gpd.read_file('./shapefiles/precise_villages.gpkg', layer=country)
        # gdf=gpd.GeoDataFrame(gdf.drop(columns='geometry'), geometry=gdf.geometry.centroid, crs=gdf.crs) # get village centroids
    else:
        gdf=gpd.read_file('pals/shapefiles1/precise_villages.gpkg', layer=country)
        #gdf=gpd.read_file(path, layer=country)
    # convert gdf to ee feature collection
    roi=(ee.FeatureCollection(geemap.gdf_to_ee(gdf))\
         .set('country', country))

    # generate image collection for the study period and apply functions
    if dataset=='landsat':
        # masks out clouds
        def mask_clouds(image):
            # landsat quality assesment band
            qaBand=image.select('QA_PIXEL')
            # bits 5 and 3 are cloud and cloud shadow respectively
            cloudBitMask=1<<5
            cloudShadowBitMask=1<<3
            # both bits should be equal to zero indicating clear consitions
            mask=(qaBand.bitwiseAnd(cloudBitMask).eq(0)\
                .And(qaBand.bitwiseAnd(cloudShadowBitMask).eq(0)))
            # apply the mask to the optical and thermal bands
            return (image.updateMask(mask)\
                    .select('SR_B.*', 'ST_B10')
                    .copyProperties(image, ["system:time_start"]))

        # applies landsat scaling factors
        def scale_image(image):
            opticalBands=image.select('SR_B.*').multiply(0.0000275).add(-0.2)
            thermalBand=image.select('ST_B10').multiply(0.00341802).add(149.0).subtract(273.15).rename('lst')
            return (image.addBands(opticalBands, None, True)\
                    .addBands(thermalBand, None, True))

        # computes Normalised Difference Vegetation Index
        def calculate_ndvi(image):
            ndvi=image.normalizedDifference(['SR_B5', 'SR_B4']).rename('ndvi')
            return image.addBands(ndvi)

        collection=(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
                    .filterBounds(roi)
                    .filterDate(start, end) #('2018-11-01', '2018-12-31') this period illustrates clouds for mozambique
                    .map(mask_clouds)
                    .map(scale_image)
                    .map(calculate_ndvi))

    elif dataset=='landsat-7':
        # masks out clouds
        def mask_clouds(image):
            # landsat quality assesment band
            qaBand=image.select('QA_PIXEL')
            # bits 5 and 3 are cloud and cloud shadow respectively
            cloudBitMask=1<<5
            cloudShadowBitMask=1<<3
            # both bits should be equal to zero indicating clear consitions
            mask=(qaBand.bitwiseAnd(cloudBitMask).eq(0)\
                .And(qaBand.bitwiseAnd(cloudShadowBitMask).eq(0)))
            # apply the mask to the optical and thermal bands
            return (image.updateMask(mask)\
                    .select('SR_B.*', 'ST_B6')
                    .copyProperties(image, ["system:time_start"]))

        # applies landsat scaling factors
        def scale_image(image):
            opticalBands=image.select('SR_B.*').multiply(0.0000275).add(-0.2)
            thermalBand=image.select('ST_B6').multiply(0.00341802).add(149.0).subtract(273.15).rename('lst')
            return (image.addBands(opticalBands, None, True)\
                    .addBands(thermalBand, None, True))

        # computes Normalised Difference Vegetation Index
        def calculate_ndvi(image):
            ndvi=image.normalizedDifference(['SR_B4', 'SR_B3']).rename('ndvi')
            return image.addBands(ndvi)

        collection=(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')\
                    .filterBounds(roi)
                    .filterDate(start, end)
                    .map(mask_clouds)
                    .map(scale_image)
                    .map(calculate_ndvi))

    elif dataset=='modis':
        # applies modis scaling factors
        def scale_image(image):
            if exposure=='ndvi':
                return (image.select('NDVI').multiply(0.0001).rename('ndvi')\
                        .copyProperties(image, ["system:time_start"]))
            else: # include 'LST_Night_1km' later
                return (image.select('LST_Day_1km').multiply(0.02).subtract(273.15).rename('lst')\
                        .copyProperties(image, ["system:time_start"]))
        # changing this to modis daily
        collection=ee.ImageCollection("MODIS/061/MOD13Q1") if exposure=='ndvi' else ee.ImageCollection('MODIS/061/MOD11A1')
        collection=(collection.filterBounds(roi)\
                    .filterDate(start, end)
                    .map(scale_image))

    elif dataset=='era5':
        # applies era5 scaling factors
        def scale_image(image): #change ear5 to skin temperature
            return (image.select('skin_temperature').subtract(273.15).rename('lst')\
                    .copyProperties(image, ["system:time_start"]))

        collection=(ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")\
            # ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR')\
                    .filterBounds(roi)
                    .filterDate(start, end)
                    .map(scale_image))

          #Computes the Precipitation of the ROI
    elif dataset=='persiann-cdr':

        def scale_image(image):
            return (image.select('precipitation').multiply(2.5).rename('lst')\
                    .copyProperties(image, ["system:time_start"]))


        collection=(ee.ImageCollection('NOAA/PERSIANN-CDR')\
                    .filterBounds(roi)
                    .filterDate(start,end)
                    .map(scale_image))


    elif dataset=='merra2':
            #calculate the wet bulb global temperature

            def scale_image(image):
                return (image.select('T2MWET').rename('lst')\
                        .copyProperties(image, ["system:time_start"]))

            collection=(ee.ImageCollection("NASA/GSFC/MERRA/slv/2")\
                        .filterBounds(roi)
                        .filterDate(start,end)
                        .map(scale_image))


    elif dataset=='trmm':
            #calculate the precipitation

            def scale_image(image):
                return (image.select('precipitation').rename('lst')\
                        .copyProperties(image, ["system:time_start"]))

            collection=(ee.ImageCollection("TRMM/3B43V7")\
                        .filterBounds(roi)
                        .filterDate(start,end)
                        .map(scale_image))

    return collection.select(exposure), roi

In [ ]:
# we can visualise our image collection on a map just to check
def drawCollection(collection, exposure, country):
    # map centres
    getCenter=dict(gambia=[13.443, -15.864], mozambique=[-25.1914, 32.7539], kenya=[-3.9995, 39.3609])
    # color palette
    viz={'ndvi': dict(min=-0.2, max=1, palette='8bc4f9, c9995c, c7d270, 8add60, 097210'), 'lst': dict(min=0, max=60, palette='6495ed, 32cd32, fdda0d, 8b4000, ff0000')}
    # Use folium to visualize the image collection
    map=folium.Map(location=getCenter[country], zoom_start=8)
    map.addLayer(collection[0], viz[exposure])
    map.addLayer(collection[1])
    return map

In [ ]:
def generateMonthlyTimeSeries(input_collection, exposure, country, dataset):
    # get date range of image collection
    start=ee.Date(input_collection[0].aggregate_min('system:time_start'))
    end=ee.Date(input_collection[0].aggregate_max('system:time_start'))
    n_months=end.difference(start, 'months').add(1)
    months=ee.List.sequence(0, n_months.int())
    # generate unique dates for analysis period
    dates=months.map(lambda i: start.advance(i, 'month'))

    # Groups images by month and computes mean
    def monthly_agg(date, collection):
        start=ee.Date(date)
        end=ee.Date(date).advance(1, 'month')
        collection=collection.filterDate(start, end).mean() #pixel-wise mean for entire collection
        return (collection.set('system:time_start', start.millis())\
                .set('count', collection.bandNames().length())) #this helps us identify months without images

    # generate monthly mean image collection
    mean_monthly=ee.ImageCollection.fromImages(dates.map(lambda i: monthly_agg(i, input_collection[0]))\
                                                 .filter(ee.Filter.gt('count', 0)))  #retain only non-null images

    crs_list=dict(gambia='EPSG:32628', mozambique='EPSG:32736', kenya='EPSG:32737')
    # Computes mean value for each neighborhood
    def reduceMean(image):
        features=image.reduceRegions(
            reducer=ee.Reducer.mean(),
            collection=input_collection[1],
            scale=100,
            crs=crs_list[country])
        return features.map(lambda f: f.set('exposure_month', image.date().format()))

    # Computes mean value for each health facility
    def extractPoint(image):
        features=image.sampleRegions(
            collection=input_collection[1],
            scale=100,
            geometries=True,
            projection=crs_list[country])
        return features.map(lambda f: f.set('exposure_month', image.date().format()))

    if input_collection[1].geometry().type().getInfo()[5:]=='Polygon':
        # generate monthly mean by village for image collection
        exposures=mean_monthly.map(reduceMean)
        # export to dataframe and set new index
        exposures=(geemap.ee_to_df(exposures.flatten())\
                   .rename(columns={'mean': exposure}))
        for col in exposures.columns:
            if col in ['name', 'locality', 'district', 'admin_post']:
                exposures=exposures.drop(columns=col)
        # change exposure month datetime format
        exposures['exposure_month']=exposures['exposure_month'].apply(lambda x: pd.to_datetime(x).strftime('%Y_%m'))
        return (exposures.set_index(['neighborhood_code', 'exposure_month'])\
                .sort_index())
    else:
        # extract monthly values at facility points
        exposures=mean_monthly.map(extractPoint)
        # export to dataframe and set new index
        exposures=geemap.ee_to_df(exposures.flatten())
        for col in exposures.columns:
            if col in ['name', 'nature', 'Name', 'locality', 'district', 'admin_post']:
                exposures=exposures.drop(columns=col)
        # change exposure month datetime format
        exposures['exposure_month']=exposures['exposure_month'].apply(lambda x: pd.to_datetime(x).strftime('%Y_%m'))
        return (exposures.set_index(['facility_code', 'exposure_month']) #(exposures.set_index(['neighborhood_code', 'exposure_month']) ##
                .sort_index())

In [ ]:
def generateDailyTimeSeries(input_collection, exposure, country, dataset):
    # get date range of image collection
    start=ee.Date(input_collection[0].aggregate_min('system:time_start'))
    end=ee.Date(input_collection[0].aggregate_max('system:time_start'))
    n_months=end.difference(start, 'days').add(1)
    months=ee.List.sequence(0, n_months.int())
    # generate unique dates for analysis period
    dates=months.map(lambda i: start.advance(i, 'day'))

    # Computes daily mean
    def daily_agg(date, collection):
        start=ee.Date(date)
        end=ee.Date(date).advance(1, 'day')
        collection=collection.filterDate(start, end).mean() #pixel-wise mean for entire collection
        return (collection.set('system:time_start', start.millis())\
                .set('count', collection.bandNames().length()))

    # generate daily mean image collection
    mean_daily=ee.ImageCollection.fromImages(dates.map(lambda i: daily_agg(i, input_collection[0]))\
                                                 .filter(ee.Filter.gt('count', 0)))  #retain only non-null images

    crs_list=dict(gambia='EPSG:32628', mozambique='EPSG:32736')
    # Computes mean value for each neighborhood
    def reduceMean(image):
        features=image.reduceRegions(
            reducer=ee.Reducer.mean(),
            collection=input_collection[1],
            scale=100,
            crs=crs_list[country])
        return features.map(lambda f: f.set('exposure_month', image.date().format()))

    # Computes mean value for each health facility
    def extractPoint(image):
        features=image.sampleRegions(
            collection=input_collection[1],
            scale=100,
            geometries=True,
            projection=crs_list[country])
        return features.map(lambda f: f.set('exposure_month', image.date().format()))

    if input_collection[1].geometry().type().getInfo()[5:]=='Polygon':
        # generate daily mean by village for image collection
        exposures=mean_daily.map(reduceMean)
        # export to dataframe and set new index
        exposures=(geemap.ee_to_df(exposures.flatten())\
                   .rename(columns={'mean': exposure})
                   .drop(columns='name'))
        # change exposure day datetime format
        exposures['exposure_month']=exposures['exposure_month'].apply(lambda x: pd.to_datetime(x).strftime('%Y_%m_%d'))
        return (exposures.set_index(['neighborhood_code', 'exposure_month'])\
                .sort_index())
    else:
        # extract daily values at facility points
        exposures=mean_daily.map(extractPoint)
        # export to dataframe and set new index
        exposures=(geemap.ee_to_df(exposures.flatten())\
                  .drop(columns=['name', 'nature']))
        # change exposure day datetime format
        exposures['exposure_month']=exposures['exposure_month'].apply(lambda x: pd.to_datetime(x).strftime('%Y_%m_%d'))
        return (exposures.set_index(['facility_code', 'exposure_month'])\
                .sort_index())

In [ ]:
def aggregateTimeSeries(input_collection, exposure, country, dataset):
    county_list=dict(gambia=['Central Baddibu', 'Upper Baddibu'], mozambique=['Manhica'], kenya=['Kilifi', 'Kwale'])
    # utility for filtering by county
    county_filter=[ee.Filter.eq('ADM2_NAME', county) for county in county_list[country.lower()]]
    # get admin boundaries for country
    counties=(ee.FeatureCollection("FAO/GAUL/2015/level2")\
             .filter(ee.Filter.Or(*county_filter)))

    crs_list=dict(gambia='EPSG:32628', mozambique='EPSG:32736', kenya='EPSG:32737')

    # get date range of image collection
    start=ee.Date(input_collection[0].aggregate_min('system:time_start'))
    end=ee.Date(input_collection[0].aggregate_max('system:time_start'))
    n_months=end.difference(start, 'months').add(1)
    months=ee.List.sequence(0, n_months.int())
    # generate unique dates for analysis period
    dates=months.map(lambda i: start.advance(i, 'month'))

    # Groups images by month and computes mean
    def monthly_agg(date, collection):
        start=ee.Date(date)
        end=ee.Date(date).advance(1, 'month')
        collection=collection.filterDate(start, end).mean() #pixel-wise mean for entire collection
        return (collection.set('system:time_start', start.millis())\
                .set('count', collection.bandNames().length())) #this helps us identify months without images

    # generate monthly mean image collection
    mean_monthly=ee.ImageCollection.fromImages(dates.map(lambda i: monthly_agg(i, input_collection[0]))\
                                                 .filter(ee.Filter.gt('count', 0)))  #retain only non-null images

    # Computes mean value for entire image
    def reduceMean(image):
        mean=image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=counties,
            scale=30,
            crs=crs_list[country]
        ).get(exposure)
        return image.set(exposure, mean).set('exposure_month', image.date().format())

    # generate overall monthly mean for roi
    exposures=(mean_monthly.map(reduceMean)\
            .reduceColumns(ee.Reducer.toList(2), ['exposure_month', exposure]).values().get(0))

    # export to dataframe and set new index
    exposures=pd.DataFrame(exposures.getInfo(), columns=['exposure_month', exposure])
    # change exposure month datetime format
    exposures['exposure_month']=exposures['exposure_month'].apply(lambda x: pd.to_datetime(x).strftime('%Y_%m'))

    return exposures.set_index('exposure_month')

In [ ]:
def cimatologyMaps(input_collection, exposure, country, dataset):
    county_list=dict(gambia=['Central Baddibu', 'Upper Baddibu'], mozambique=['Manhica'], kenya=['Kilifi', 'Kwale'])
    # utility for filtering by county
    county_filter=[ee.Filter.eq('ADM2_NAME', county) for county in county_list[country.lower()]]
    # get admin boundaries for country
    counties=(ee.FeatureCollection("FAO/GAUL/2015/level2")\
             .filter(ee.Filter.Or(*county_filter)))

    crs_list=dict(gambia='EPSG:32628', mozambique='EPSG:32736', kenya='EPSG:32737')

    # get month range of a year
    months=ee.List.sequence(1, 12)

    def monthly_agg(m):
        return (input_collection[0].filter(ee.Filter.calendarRange(m, m, 'month'))\
                .mean()
                .set('month', m))

    # group by month and calculate monthly mean image collection
    mean_monthly=(ee.ImageCollection.fromImages(months.map(monthly_agg))\
                  .map(lambda i: i.clip(counties)))

    # export image to xarray
    lst_raster=geemap.ee_to_xarray(mean_monthly, crs='EPSG:4326', scale=30*0.001/111, geometry=counties.geometry())
    # specify nodata value
    lst_raster['lst']=(lst_raster['lst'].rio.write_nodata(np.nan)
                # specify spatial dimensions
                .rio.set_spatial_dims('lon', 'lat'))
    return lst_raster

In [ ]:
def monthly_exposures(exposure, country, dataset, facilities=False, draw_map=False):
    images=generateImageCollection(exposure, country, dataset, facilities)
    return drawCollection(images, exposure, country) if draw_map else generateMonthlyTimeSeries(images, exposure, country, dataset)

In [ ]:
def daily_exposures(exposure, country, dataset, facilities=False, draw_map=False):
    images=generateImageCollection(exposure, country, dataset, facilities)
    return drawCollection(images, exposure, country) if draw_map else generateDailyTimeSeries(images, exposure, country, dataset)

In [ ]:
def agg_monthly_exposures(exposure, country, dataset, facilities=False, draw_map=False):
    images=generateImageCollection(exposure, country, dataset, facilities)
    return drawCollection(images, exposure, country) if draw_map else aggregateTimeSeries(images, exposure, country, dataset)

In [ ]:
def agg_monthly_exposure_maps(exposure, country, dataset, facilities=False, draw_map=False):
    images=generateImageCollection(exposure, country, dataset, facilities)
    return cimatologyMaps(images, exposure, country, dataset)


# **Generate exposures and export**

In [ ]:
lsat=monthly_exposures(exposure='lst', country='kenya', dataset='landsat', facilities=False)
lsat7=monthly_exposures(exposure='lst', country='kenya', dataset='landsat-7', facilities=False)
modis=monthly_exposures(exposure='lst', country='kenya', dataset='modis', facilities=False)
era5=monthly_exposures(exposure='lst', country='kenya', dataset='era5', facilities=False)
persiann=monthly_exposures(exposure='lst', country='kenya', dataset='persiann-cdr', facilities=False)
#merra2=monthly_exposures(exposure='lst', country='gambia', dataset='merra2', facilities=False)
trmm=monthly_exposures(exposure='lst', country='kenya', dataset='trmm', facilities=False)

In [ ]:
lsat#.groupby(by=lsat.index).agg({'exposure':'max'})
lsat7
modis
era5
persiann
merra2
trmm

lst
neighborhood_code exposure_month           
3                 2018_11         42.873081
                  2018_12         41.676736
                  2019_01         36.889391
                  2019_02         41.193383
                  2019_03         47.764415
...                                     ...
277               2022_08         42.481266
                  2022_09               NaN
                  2022_10         40.287336
                  2022_11         45.353388
                  2022_12         38.025714

[4600 rows x 1 columns]

In [ ]:

# Assuming lsat, lsat7, modis, era5, trmm, and persiann are already defined

# Rename columns
lsat = lsat.rename(columns={'lst': 'landsat'})
lsat7 = lsat7.rename(columns={'lst': 'landsat-7'})
modis = modis.rename(columns={'lst': 'modis'})
era5 = era5.rename(columns={'lst': 'era5'})
trmm = trmm.rename(columns={'lst': 'trmm'})
persiann = persiann.rename(columns={'lst': 'persiann'})

# Remove duplicates based on index
lsat = lsat[~lsat.index.duplicated(keep='first')]
lsat7 = lsat7[~lsat7.index.duplicated(keep='first')]
modis = modis[~modis.index.duplicated(keep='first')]
era5 = era5[~era5.index.duplicated(keep='first')]
trmm = trmm[~trmm.index.duplicated(keep='first')]
persiann = persiann[~persiann.index.duplicated(keep='first')]

# Merge the DataFrames
lst = pd.merge(lsat, modis, left_index=True, right_index=True, how='outer', validate='1:1')
lst = pd.merge(lst, era5, left_index=True, right_index=True, how='outer', validate='1:1')
lst = pd.merge(lst, trmm, left_index=True, right_index=True, how='outer', validate='1:1')
lst = pd.merge(lst, persiann, left_index=True, right_index=True, how='outer', validate='1:1')
lst = pd.merge(lst, lsat7, left_index=True, right_index=True, how='outer', validate='1:1')

# Select the columns you need
lst = lst[['landsat', 'modis', 'era5', 'trmm', 'persiann', 'landsat-7']]

print(lst)

MergeError: Passing 'suffixes' which cause duplicate columns {'accuracy_x', 'longitude_x', 'chu_x', 'Shape_Leng_x', 'village_co_x', 'BUFF_DIST_x', 'ORIG_FID_2_x', 'OBJECTID_1_x', 'ORIG_FID_x', 'ORIG_FID_1_x', 'OBJECTID_x', 'Shape_Area_x', 'latitude_x'} is not allowed.

In [ ]:
lsat=lsat.rename(columns={'lst': 'landsat'})
lsat7=lsat7.rename(columns={'lst': 'landsat-7'})
modis=modis.rename(columns={'lst': 'modis'})
era5=era5.rename(columns={'lst': 'era5'})
trmm=trmm.rename(columns={'lst': 'trmm'})
persiann=persiann.rename(columns={'lst': 'persiann'})

lst=pd.merge(lsat, modis, left_index=True, right_index=True, how='outer', validate='1:1')
lst=pd.merge(lst, era5, left_index=True, right_index=True, how='outer', validate='1:1')
lst=pd.merge(lst, trmm, left_index=True, right_index=True, how='outer',suffixes=('_left','_right'), validate='1:1')
lst=pd.merge(lst, persiann, left_index=True, right_index=True, how='outer', validate='1:1')
lst=pd.merge(lst, lsat7, left_index=True, right_index=True, how='outer',suffixes=('_left1','_right1'), validate='1:1')[['landsat', 'modis', 'era5', 'trmm','persiann', 'landsat-7']]
lst

landsat      modis       era5      trmm  \
neighborhood_code exposure_month                                              
254001            2018_11         30.185115  35.333096  27.584496  0.047712   
                  2018_12         35.721315  31.826415  28.546576  0.091021   
                  2019_01         33.054225  34.376743  29.504550  0.014271   
                  2019_02         39.273253  35.634200  30.567800  0.023257   
                  2019_03         35.604305  39.822589  31.788465  0.008679   
...                                     ...        ...        ...       ...   
254126            2023_08               NaN  28.756062  24.953545       NaN   
                  2023_09               NaN  34.889398  26.178829       NaN   
                  2023_10         49.941434  35.866991  27.130775       NaN   
                  2023_11         39.380531  29.973363  26.032494       NaN   
                  2023_12               NaN  30.122566  27.688435       NaN   

                                   persiann  landsat-7  
neighborhood_code exposure_month                        
254001            2018_11          7.772623  28.202838  
                  2018_12          5.708913        NaN  
                  2019_01          0.592062        NaN  
                  2019_02          1.264416  32.746749  
                  2019_03          0.845130        NaN  
...                                     ...        ...  
254126            2023_08          0.000000  25.633187  
                  2023_09          0.000000  29.936588  
                  2023_10          9.774372  23.621814  
                  2023_11         39.541232  21.734057  
                  2023_12          7.639496        NaN  

[7812 rows x 6 columns]